## 전처리

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder # for encoding
from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler #for standardization
from sklearn.model_selection import train_test_split
import time

C:\Users\JungHun.DESKTOP-JRGUSS9\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\JungHun.DESKTOP-JRGUSS9\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [2]:
import plotly.graph_objects as go
import plotly.express as px
from scipy import stats
from scipy.stats import chi2_contingency

In [3]:
# competition_format
x_train = pd.read_csv('./data/Smoking_raw/competition_format/x_train.csv')
# x_test = pd.read_csv('./data/Smoking_raw/competition_format/x_test.csv')
y_train = pd.read_csv('./data/Smoking_raw/competition_format/y_train.csv')
# y_test = pd.read_csv('./data/Smoking_raw/competition_format/y_test.csv')

In [4]:
x_train.head()

,ID,gender,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),...,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,oral,dental caries,tartar
0,0,F,40,155,60,81.3,1.2,1.0,1.0,1.0,...,126.0,12.9,1.0,0.7,18.0,19.0,27.0,Y,0.0,Y
1,1,F,40,160,60,81.0,0.8,0.6,1.0,1.0,...,127.0,12.7,1.0,0.6,22.0,19.0,18.0,Y,0.0,Y
2,2,M,55,170,60,80.0,0.8,0.8,1.0,1.0,...,151.0,15.8,1.0,1.0,21.0,16.0,22.0,Y,0.0,N
3,3,M,40,165,70,88.0,1.5,1.5,1.0,1.0,...,226.0,14.7,1.0,1.0,19.0,26.0,18.0,Y,0.0,Y
4,4,F,40,155,60,86.0,1.0,1.0,1.0,1.0,...,107.0,12.5,1.0,0.6,16.0,14.0,22.0,Y,0.0,N


In [5]:
y_train.head()

,ID,smoking
0,0,0
1,1,0
2,2,1
3,3,0
4,4,0


In [6]:
train=x_train.merge(y_train,on='ID')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44553 entries, 0 to 44552
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   44553 non-null  int64  
 1   gender               44553 non-null  object 
 2   age                  44553 non-null  int64  
 3   height(cm)           44553 non-null  int64  
 4   weight(kg)           44553 non-null  int64  
 5   waist(cm)            44553 non-null  float64
 6   eyesight(left)       44553 non-null  float64
 7   eyesight(right)      44553 non-null  float64
 8   hearing(left)        44553 non-null  float64
 9   hearing(right)       44553 non-null  float64
 10  systolic             44553 non-null  float64
 11  relaxation           44553 non-null  float64
 12  fasting blood sugar  44553 non-null  float64
 13  Cholesterol          44553 non-null  float64
 14  triglyceride         44553 non-null  float64
 15  HDL                  44553 non-null 

가설 검정 변수

연속형 변수 : HDL,LDL,hemoglobin,triglyceride,fasting blood sugar,Gtp ( T검정으로 검정)


범주형 변수 : dental caries, tartar (카이제곱으로 검정)



In [7]:
cont_var=['HDL','LDL','hemoglobin','triglyceride','fasting blood sugar','Gtp']
cate_var=['dental caries','tartar']

# T-test를 위한 연속형 변수 데이터 스플릿

In [8]:
def group_split(df,var):
    smoker=df[df['smoking']==1][var]
    nonsmoker=df[df['smoking']==0][var]
    
    return smoker,nonsmoker

# Chi-Test를 위한 범주형 변수 교차 테이블 생성

In [45]:
def piv(df,var):
    result=df.pivot_table(index='smoking',columns=var,aggfunc='size')
    result.index=['non-smoker','smoker']
    result.columns=['n','y']
    return result

# var_type에 따라 검정방법을 다르게하여 검정하는 함수
**- var_type이 continuous면 연속형 변수로 T-test진행**  
    1. 등분산 검정결과에 따라 t-test의 등분산을 가정할 것인지 판단  
    2. equal_var=True면 분산이 같다고 판단하여 t-test진행/False이면 분산이 다르다고 판단하여 t-test진행  
    

**- var_type이 continuous가 아니면 범주형 변수로 Chi-test 진행**

In [10]:
# 연속형 변수와 범주형 변수로 나누어서 검정
def test(var,var_type='continuous'):
    smoker,nonsmoker=group_split(train,var)
    if var_type=='continuous':
        # 분산 동질성 검정
        test_res1=stats.levene(smoker, nonsmoker)
        
        # 동질성 검정 결과에 따라 동분산가정 T-test여부 결정
        if test_res1[1]>=0.05:
            t_stat, p_value = stats.ttest_ind(smoker, nonsmoker, equal_var=True)
            print('P-value:', p_value)
            if p_value>=0.05:
                print('귀무가설 채택 : p값이 0.05보다 크므로 흡연여부에 따라 통계적으로 차이가 없음.')
            else :
                print('대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 차이가 있음.')
        else :
            t_stat, p_value = stats.ttest_ind(smoker, nonsmoker, equal_var=False)
            print('P-value:', p_value)
            if p_value>=0.05:
                print('귀무가설 채택 : p값이 0.05보다 크므로 흡연여부에 따라 통계적으로 차이가 없음.')
            else :
                print('대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 차이가 있음.')
    else : 
        observed=piv(train,var)
        chi,p_value,dof,exp=chi2_contingency(observed)
        print('P-value:',p_value)
        if p_value>=0.5:
            print('귀무가설 채택 : p값이 0.05보다 크므로 흡연여부에 따라 통계적으로 연관성이 없음.')
        else :
            print('대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 연관성이 있음.')

# 연속형 변수 가설검정

In [12]:
for i in cont_var:
    print(i)
    test(i)
    print('\n')

HDL
P-value: 3.20425221812743e-309
대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 차이가 있음.


LDL
P-value: 2.9618249246658e-20
대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 차이가 있음.


hemoglobin
P-value: 0.0
대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 차이가 있음.


triglyceride
P-value: 0.0
대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 차이가 있음.


fasting blood sugar
P-value: 4.564077325980962e-84
대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 차이가 있음.


Gtp
P-value: 0.0
대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 차이가 있음.




In [83]:
# 효과크기 직접계산
def cohens_d(group1, group2):
    mean1, mean2 = np.mean(group1), np.mean(group2)
    std1, std2 = np.std(group1, ddof=1), np.std(group2, ddof=1)
    pooled_std = np.sqrt(((len(group1) - 1) * std1**2 + (len(group2) - 1) * std2**2) / (len(group1) + len(group2) - 2))
    cohens_d=(mean1 - mean2) / pooled_std
    return cohens_d

In [86]:
for i in range(len(cont_var)):
    smoker,nonsmoker=group_split(train,cont_var[i])
    print(f"{cont_var[i]} cohen's d : {cohens_d(smoker,nonsmoker)}")
    


HDL cohen's d : -0.3707052642080041
LDL cohen's d : -0.09347475597439728
hemoglobin cohen's d : 0.8981989212025849
triglyceride cohen's d : 0.5377339713131515
fasting blood sugar cohen's d : 0.20726418291531942
Gtp cohen's d : 0.5014602936469544


# 범주형 변수 가설검정

In [11]:
for i in cate_var:
    print(i)
    test(i,var_type='category')
    print('\n')

dental caries
P-value: 2.682243235128026e-109
대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 차이가 있음.


tartar
P-value: 4.245470858017243e-104
대립가설 채택 : p값이 0.05보다 작으므로 흡연여부에 따라 통계적으로 차이가 있음.




In [67]:
def cramer(cr_df):
    n = cr_df.values.sum()  # 전체 샘플 수
    chi2, p, dof, expected = chi2_contingency(cr_df)
    cramers_v = np.sqrt(chi2 / (n * min(cr_df.shape[0] - 1, cr_df.shape[1] - 1)))
    print(f"{cate_var[i]} Cramér's V: {cramers_v}")

In [68]:
for i in range(len(cate_var)):
    df=piv(train,cate_var[i])
    cramer(df)
    

dental caries Cramér's V: 0.10522811762768798
tartar Cramér's V: 0.10264810555531649


# 효과크기 계산 라이브러리 예시
(직접 계산하였으나 라이브러리도 존재)

In [3]:
import pingouin

C:\Users\JungHun.DESKTOP-JRGUSS9\AppData\Roaming\Python\Python39\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.8.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
C:\Users\JungHun.DESKTOP-JRGUSS9\AppData\Roaming\Python\Python39\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.4' currently installed).
  from pandas.core import (


In [25]:
pingouin.compute_effsize(smoker, nonsmoker, eftype='cohen')

0.5014602936469544